# Milestone 1: Data Acquisition and Exploration

By Eric Zack and Daniel Nour

## Research Question
How accurate were the 500mb geopotential heights over a day or two to a month?

## Objective
This project aims to verify the accuracy of 500mb geopotential heights from the 
ECMWF operational model by comparing forecasted values against verification data over varying time frames.

## Data Acquisition

### Data Source
We utilized the ECMWF (European Centre for Medium-Range Weather Forecasts) operational model data accessed through the Herbie library. Specifically, we retrieved data related to the 500mb geopotential height (`gh`).

### Data Retrieval
- **Model:** Integrated Forcast System (IFS)
- **Product:** Operational (`oper`)
- **Forecast Hours:** 0, 72, 144, 240
- **Time Frame:** October 1, 2024, to October 11, 2024 (every 6 hours)

### Data Storage
The downloaded data is stored locally in the specified directory on our server. Ensure that the path is correctly set in your environment to access the data seamlessly.

In [2]:
# Import necessary Libraries
from herbie import Herbie, FastHerbie
import pandas as pd
import numpy as np
import os
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import cdsapi 
import shutil

## DATA EXPLORATION

In this section, we explore the raw data by generating preliminary plots of the 500mb geopotential heights at various forecast hours. These plots help us understand the spatial distribution and any apparent anomalies in the data

In [2]:
run_date = "2024-10-02"
forecast_hours = [0, 72, 144, 240]

timelist = pd.date_range(start='2024-10-01', end='2024-10-11', freq='6H')
H = FastHerbie(timelist, model="ifs", product="oper", fxx=[0])
Hforecast0 = FastHerbie(timelist[:1], model="ifs", product="oper", fxx=forecast_hours, max_threads=2)

df = H.inventory()
df.reference_time.unique()

/tmp/ipykernel_65487/3825717747.py:4: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  timelist = pd.date_range(start='2024-10-01', end='2024-10-11', freq='6H')
Could not find 20/21 GRIB files.


<DatetimeArray>
['2024-10-01 00:00:00', '2024-10-01 12:00:00', '2024-10-02 00:00:00',
 '2024-10-02 12:00:00', '2024-10-03 00:00:00', '2024-10-03 12:00:00',
 '2024-10-04 00:00:00', '2024-10-04 12:00:00', '2024-10-05 00:00:00',
 '2024-10-05 12:00:00', '2024-10-06 00:00:00', '2024-10-06 12:00:00',
 '2024-10-07 00:00:00', '2024-10-07 12:00:00', '2024-10-08 00:00:00',
 '2024-10-08 12:00:00', '2024-10-09 00:00:00', '2024-10-09 12:00:00',
 '2024-10-10 00:00:00', '2024-10-10 12:00:00', '2024-10-11 00:00:00']
Length: 21, dtype: datetime64[ns]

In [ ]:
ss = r":gh:500:"

ds2 = Hforecast0.xarray(ss)
ds2 = ds2.sel(longitude=slice(-130, -60), latitude=slice(60, 20))
ghforecast = ds2['gh']

lat = ghforecast.latitude.values
lon = ghforecast.longitude.values
dataproj = ccrs.PlateCarree()

ghforecast0 = ghforecast[0]
ghforecast72 = ghforecast[1]
ghforecast144 = ghforecast[2]
ghforecast240 = ghforecast[3]

## PLOTS

In [ ]:
ghinsert = ghforecast0

fig = plt.figure(figsize=(12,9))
ax = fig.add_subplot(111, projection=ccrs.LambertConformal())
ax.set_extent([-117,-75,20,55], ccrs.PlateCarree())
ax.add_feature(cfeature.COASTLINE.with_scale('50m'))
ax.add_feature(cfeature.STATES.with_scale('50m'), linestyle=':')
ax.add_feature(cfeature.BORDERS.with_scale('50m'))

cs = ax.contour(lon, lat, ghinsert, colors='k', transform=dataproj)
plt.clabel(cs)
ax.contourf(lon, lat, ghinsert, cmap = 'RdYlGn_r' ,transform=dataproj)
color_map = ax.contourf(lon, lat, ghinsert, cmap = 'RdYlGn_r' ,transform=dataproj)
ax.set_title('Geopotential Height Forecast (m), October 1, 2024')
plt.colorbar(color_map, ax=ax)

ghinsert = ghforecast72
#ghplot = ghinsert.values[0,:,:]
#ghplot

fig = plt.figure(figsize=(12,9))
ax = fig.add_subplot(111, projection=ccrs.LambertConformal())
ax.set_extent([-117,-75,20,55], ccrs.PlateCarree())
ax.add_feature(cfeature.COASTLINE.with_scale('50m'))
ax.add_feature(cfeature.STATES.with_scale('50m'), linestyle=':')
ax.add_feature(cfeature.BORDERS.with_scale('50m'))

# Plot parameters
cs = ax.contour(lon, lat, ghinsert, colors='k', transform=dataproj)
plt.clabel(cs)
ax.contourf(lon, lat, ghinsert, cmap = 'RdYlGn_r' ,transform=dataproj)
color_map = ax.contourf(lon, lat, ghinsert, cmap = 'RdYlGn_r' ,transform=dataproj)
ax.set_title('Geopotential Height Forecast (m), October 4, 2024')
plt.colorbar(color_map, ax=ax)

ghinsert = ghforecast144
#ghplot = ghinsert.values[0,:,:]
#ghplot

fig = plt.figure(figsize=(12,9))
ax = fig.add_subplot(111, projection=ccrs.LambertConformal())
ax.set_extent([-117,-75,20,55], ccrs.PlateCarree())
ax.add_feature(cfeature.COASTLINE.with_scale('50m'))
ax.add_feature(cfeature.STATES.with_scale('50m'), linestyle=':')
ax.add_feature(cfeature.BORDERS.with_scale('50m'))

# Plot parameters
cs = ax.contour(lon, lat, ghinsert, colors='k', transform=dataproj)
plt.clabel(cs)
ax.contourf(lon, lat, ghinsert, cmap = 'RdYlGn_r' ,transform=dataproj)
color_map = ax.contourf(lon, lat, ghinsert, cmap = 'RdYlGn_r' ,transform=dataproj)
ax.set_title('Geopotential Height Forecast (m), October 7, 2024')
plt.colorbar(color_map, ax=ax)

ghinsert = ghforecast240
#ghplot = ghinsert.values[0,:,:]
#ghplot

fig = plt.figure(figsize=(12,9))
ax = fig.add_subplot(111, projection=ccrs.LambertConformal())
ax.set_extent([-117,-75,20,55], ccrs.PlateCarree())
ax.add_feature(cfeature.COASTLINE.with_scale('50m'))
ax.add_feature(cfeature.STATES.with_scale('50m'), linestyle=':')
ax.add_feature(cfeature.BORDERS.with_scale('50m'))

# Plot parameters
cs = ax.contour(lon, lat, ghinsert, colors='k', transform=dataproj)
plt.clabel(cs)
ax.contourf(lon, lat, ghinsert, cmap = 'RdYlGn_r' ,transform=dataproj)
color_map = ax.contourf(lon, lat, ghinsert, cmap = 'RdYlGn_r' ,transform=dataproj)
ax.set_title('Geopotential Height Forecast (m), October 10, 2024')
plt.colorbar(color_map, ax=ax)

## Verification Maps
Similar to the forecast maps, we plotted the verification data to compare against the forecasts.

In [ ]:
ds = H.xarray(ss)
ds = ds.sortby(['latitude', 'longitude'])
ds = ds.sel(longitude=slice(-130, -60), latitude=slice(20,60))
timeplot = ds['time']

gh = ds['gh']
gh0 = gh[0]
gh72 = gh[6]
gh144 = gh[12]
gh240 = gh[18]
gh = gh.sortby(['time'])

lat = gh.latitude.values
lon = gh.longitude.values
dataproj = ccrs.PlateCarree()

In [ ]:
ghinsert = gh0
#ghplot = ghinsert.values[0,:,:]
#ghplot

fig = plt.figure(figsize=(12,9))
ax = fig.add_subplot(111, projection=ccrs.LambertConformal())
ax.set_extent([-117,-75,20,55], ccrs.PlateCarree())
ax.add_feature(cfeature.COASTLINE.with_scale('50m'))
ax.add_feature(cfeature.STATES.with_scale('50m'), linestyle=':')
ax.add_feature(cfeature.BORDERS.with_scale('50m'))

# Plot parameters
cs = ax.contour(lon, lat, ghinsert, colors='k', transform=dataproj)
plt.clabel(cs)
ax.contourf(lon, lat, ghinsert, cmap = 'RdYlGn_r' ,transform=dataproj)
color_map = ax.contourf(lon, lat, ghinsert, cmap = 'RdYlGn_r' ,transform=dataproj)
ax.set_title('Geopotential Height Verification (m), October 1, 2024')
plt.colorbar(color_map, ax=ax)


ghinsert = gh72
#ghplot = ghinsert.values[0,:,:]
#ghplot

fig = plt.figure(figsize=(12,9))
ax = fig.add_subplot(111, projection=ccrs.LambertConformal())
ax.set_extent([-117,-75,20,55], ccrs.PlateCarree())
ax.add_feature(cfeature.COASTLINE.with_scale('50m'))
ax.add_feature(cfeature.STATES.with_scale('50m'), linestyle=':')
ax.add_feature(cfeature.BORDERS.with_scale('50m'))

# Plot parameters
cs = ax.contour(lon, lat, ghinsert, colors='k', transform=dataproj)
plt.clabel(cs)
ax.contourf(lon, lat, ghinsert, cmap = 'RdYlGn_r' ,transform=dataproj)
color_map = ax.contourf(lon, lat, ghinsert, cmap = 'RdYlGn_r' ,transform=dataproj)
ax.set_title('Geopotential Height Verification (m), October 4, 2024')
plt.colorbar(color_map, ax=ax)

ghinsert = gh144
#ghplot = ghinsert.values[0,:,:]
#ghplot

fig = plt.figure(figsize=(12,9))
ax = fig.add_subplot(111, projection=ccrs.LambertConformal())
ax.set_extent([-117,-75,20,55], ccrs.PlateCarree())
ax.add_feature(cfeature.COASTLINE.with_scale('50m'))
ax.add_feature(cfeature.STATES.with_scale('50m'), linestyle=':')
ax.add_feature(cfeature.BORDERS.with_scale('50m'))

# Plot parameters
cs = ax.contour(lon, lat, ghinsert, colors='k', transform=dataproj)
plt.clabel(cs)
ax.contourf(lon, lat, ghinsert, cmap = 'RdYlGn_r' ,transform=dataproj)
color_map = ax.contourf(lon, lat, ghinsert, cmap = 'RdYlGn_r' ,transform=dataproj)
ax.set_title('Geopotential Height Verification (m), October 7, 2024')
plt.colorbar(color_map, ax=ax)

ghinsert = gh240
#ghplot = ghinsert.values[0,:,:]
#ghplot

fig = plt.figure(figsize=(12,9))
ax = fig.add_subplot(111, projection=ccrs.LambertConformal())
ax.set_extent([-117,-75,20,55], ccrs.PlateCarree())
ax.add_feature(cfeature.COASTLINE.with_scale('50m'))
ax.add_feature(cfeature.STATES.with_scale('50m'), linestyle=':')
ax.add_feature(cfeature.BORDERS.with_scale('50m'))

# Plot parameters
cs = ax.contour(lon, lat, ghinsert, colors='k', transform=dataproj)
plt.clabel(cs)
ax.contourf(lon, lat, ghinsert, cmap = 'RdYlGn_r' ,transform=dataproj)
color_map = ax.contourf(lon, lat, ghinsert, cmap = 'RdYlGn_r' ,transform=dataproj)
ax.set_title('Geopotential Height Verification (m), October 10, 2024')
plt.colorbar(color_map, ax=ax)

## FINDING THE DIFFERENCE PLOTS
By calculating the differences between the forecasted and verification geopotential heights, we generated difference maps. These maps highlight areas where the model performed well and regions where there were significant errors.

In [ ]:
gh0dif= gh0-ghforecast0
gh72dif= gh72-ghforecast72
gh144dif= gh144-ghforecast144
gh240dif= gh240-ghforecast240

ghinsert = gh0dif

fig = plt.figure(figsize=(12,9))
ax = fig.add_subplot(111, projection=ccrs.LambertConformal())
ax.set_extent([-117,-75,20,55], ccrs.PlateCarree())
ax.add_feature(cfeature.COASTLINE.with_scale('50m'))
ax.add_feature(cfeature.STATES.with_scale('50m'), linestyle=':')
ax.add_feature(cfeature.BORDERS.with_scale('50m'))

cs = ax.contour(lon, lat, ghinsert, colors='k', transform=dataproj)
plt.clabel(cs)
ax.contourf(lon, lat, ghinsert, cmap = 'RdYlGn_r' ,transform=dataproj)
color_map = ax.contourf(lon, lat, ghinsert, cmap = 'RdYlGn_r' ,transform=dataproj)
ax.set_title('Geopotential Height Difference (m), October 1, 2024')
plt.colorbar(color_map, ax=ax)

ghinsert = gh72dif

fig = plt.figure(figsize=(12,9))
ax = fig.add_subplot(111, projection=ccrs.LambertConformal())
ax.set_extent([-117,-75,20,55], ccrs.PlateCarree())
ax.add_feature(cfeature.COASTLINE.with_scale('50m'))
ax.add_feature(cfeature.STATES.with_scale('50m'), linestyle=':')
ax.add_feature(cfeature.BORDERS.with_scale('50m'))

cs = ax.contour(lon, lat, ghinsert, colors='k', transform=dataproj)
plt.clabel(cs)
ax.contourf(lon, lat, ghinsert, cmap = 'RdYlGn_r' ,transform=dataproj)
color_map = ax.contourf(lon, lat, ghinsert, cmap = 'RdYlGn_r' ,transform=dataproj)
ax.set_title('Geopotential Height Difference (m), October 4, 2024')
plt.colorbar(color_map, ax=ax)

ghinsert = gh144dif

fig = plt.figure(figsize=(12,9))
ax = fig.add_subplot(111, projection=ccrs.LambertConformal())
ax.set_extent([-117,-75,20,55], ccrs.PlateCarree())
ax.add_feature(cfeature.COASTLINE.with_scale('50m'))
ax.add_feature(cfeature.STATES.with_scale('50m'), linestyle=':')
ax.add_feature(cfeature.BORDERS.with_scale('50m'))


cs = ax.contour(lon, lat, ghinsert, colors='k', transform=dataproj)
plt.clabel(cs)
ax.contourf(lon, lat, ghinsert, cmap = 'RdYlGn_r' ,transform=dataproj)
color_map = ax.contourf(lon, lat, ghinsert, cmap = 'RdYlGn_r' ,transform=dataproj)
ax.set_title('Geopotential Height Difference (m), October 7, 2024')
plt.colorbar(color_map, ax=ax)

ghinsert = gh240dif

fig = plt.figure(figsize=(12,9))
ax = fig.add_subplot(111, projection=ccrs.LambertConformal())
ax.set_extent([-117,-75,20,55], ccrs.PlateCarree())
ax.add_feature(cfeature.COASTLINE.with_scale('50m'))
ax.add_feature(cfeature.STATES.with_scale('50m'), linestyle=':')
ax.add_feature(cfeature.BORDERS.with_scale('50m'))

cs = ax.contour(lon, lat, ghinsert, colors='k', transform=dataproj)
plt.clabel(cs)
ax.contourf(lon, lat, ghinsert, cmap = 'RdYlGn_r' ,transform=dataproj)
color_map = ax.contourf(lon, lat, ghinsert, cmap = 'RdYlGn_r' ,transform=dataproj)
ax.set_title('Geopotential Height Difference (m), October 10, 2024')
plt.colorbar(color_map, ax=ax)

## Custom Colormap Testing with Plots
To make the differences easier to see, we used a custom color map that highlights where the discrepancies are positive or negative.

In [ ]:
custom_cmap = ListedColormap(['white', 'peachpuff', 'sandybrown', 'peru', 'sienna'])

ghinsert = gh240dif
#ghplot = ghinsert.values[0,:,:]
#ghplot

fig = plt.figure(figsize=(12,9))
ax = fig.add_subplot(111, projection=ccrs.LambertConformal())
ax.set_extent([-117,-75,20,55], ccrs.PlateCarree())
ax.add_feature(cfeature.COASTLINE.with_scale('50m'))
ax.add_feature(cfeature.STATES.with_scale('50m'), linestyle=':')
ax.add_feature(cfeature.BORDERS.with_scale('50m'))

# Plot parameters
cs = ax.contour(lon, lat, ghinsert, colors='k', transform=dataproj)
plt.clabel(cs)
ax.contourf(lon, lat, ghinsert, cmap = custom_cmap ,transform=dataproj)
color_map = ax.contourf(lon, lat, ghinsert, cmap = custom_cmap ,transform=dataproj)
ax.set_title('Geopotential Height Difference (m), October 10, 2024')
plt.colorbar(color_map, ax=ax)

## ERA 5
We used the FastHerbie class to automate the download of ECMWF model runs. By specifying the run date and forecast hours (0, 72, 144, and 240), the function fetches the necessary GRIB2 files and saves them to our designated directory: /home/meteo/dvn5267/meteo473_groupwork/group9/data/ecmwf.

To download the ERA5 data, we created a function called download_era5. This function uses the CDS API to request the 500mb geopotential height data for a specific date and saves it to our designated folder.

In [ ]:
model_data_dir = "/home/meteo/dvn5267/meteo473_groupwork/group9/data/ecmwf"
era5_data_dir = "/home/meteo/dvn5267/meteo473_groupwork/group9/data/era5"

os.makedirs(model_data_dir, exist_ok=True)
os.makedirs(era5_data_dir, exist_ok=True)

print(f"ECMWF data will be saved to: {model_data_dir}")
print(f"ERA5 data will be saved to: {era5_data_dir}")

In [ ]:
# ECMWF Data Download Function
def download_ecmwf_data(run_date, forecast_hours, model="ifs", product="oper", save_dir=model_data_dir, overwrite=True):
    os.makedirs(save_dir, exist_ok=True)
    base_time = pd.to_datetime(run_date)
    timelist = [base_time]
    
    H = FastHerbie(timelist, model=model, product=product, fxx=forecast_hours)
    variable = ":gh:500:"
    downloaded_files = H.download(variable)
    print(f"ECMWF data for run date {run_date} downloaded successfully.")
    
    for file_path in downloaded_files:
        dest_path = os.path.join(save_dir, os.path.basename(file_path))
        if os.path.exists(dest_path):
            if overwrite:
                os.remove(dest_path)
                shutil.move(file_path, save_dir)
                print(f"Overwrote existing file in {dest_path}")
            else:
                print(f"Skipping {file_path}, already exists in destination")
                os.remove(file_path)  # Clean up downloaded file
        else:
            shutil.move(file_path, save_dir)
            print(f"Moved {file_path} to {save_dir}")
    
    return H

In [ ]:
run_date = "2024-10-01"
forecast_hours = [0, 72, 144, 240]
download_ecmwf_data(run_date, forecast_hours)

In [1]:
def download_era5(run_date, save_dir=era5_data_dir):
    c = cdsapi.Client()
    print("CDS API client initialized successfully.")

    year = run_date[:4]
    month = run_date[5:7]
    day = run_date[8:10]

    request = {
        'product_type': 'reanalysis',
        'format': 'grib',
        'variable': [
            'geopotential',
        ],
        'pressure_level': '500',
        'year': year,
        'month': month,
        'day': day,
        'time': [
            '00:00', '06:00', '12:00', '18:00',
        ],
        'area': [60, -130, 20, -60],  # North, West, South, East
    }

    target = f"{save_dir}/ERA5_gh500_{run_date.replace('-', '')}.grib"
    c.retrieve('reanalysis-era5-pressure-levels', request, target)


In [ ]:
def load_era5_data(run_date, era5_data_dir):
    file_path = f"{era5_data_dir}/ERA5_gh500_{run_date.replace('-', '')}.grib"
    ds = xr.open_dataset(file_path, engine='cfgrib')
    return ds

In [ ]:
download_era5("2024-10-01", era5_data_dir)

In [ ]:
def regrid_data(source_ds, target_ds, method='linear'):
    target_lon = target_ds.longitude
    target_lat = target_ds.latitude
    
    if 'time' in source_ds.coords:
        source_ds = source_ds.isel(time=0)
    
    return source_ds.interp(
        longitude=target_lon,
        latitude=target_lat,
        method=method
    )

In [ ]:
def verify_with_era5(run_date, forecast_hours, era5_data_dir):
    base_time = pd.to_datetime(run_date)
    timelist = pd.date_range(start=base_time, periods=1)
    
    H = FastHerbie(timelist, model="ifs", product="oper", fxx=forecast_hours, max_threads=2)
    ds2 = H.xarray(r":gh:500:")
    ds2 = ds2.sel(longitude=slice(-130, -60), latitude=slice(60, 20))
    ghforecast = ds2['gh']

    download_era5(run_date, era5_data_dir)
    era5_ds = load_era5_data(run_date, era5_data_dir)
    
    ghforecast0 = ghforecast.isel(step=0)
    ghforecast72 = ghforecast.isel(step=1)
    ghforecast144 = ghforecast.isel(step=2)
    ghforecast240 = ghforecast.isel(step=3)
    
    era5_gh = era5_ds['z'] / 9.80665 if 'z' in era5_ds else era5_ds['gh']
    era5_regridded = regrid_data(era5_gh, ds2)
    
    differences = [
        (era5_regridded - ghforecast0, "October 1, 2024"),
        (era5_regridded - ghforecast72, "October 4, 2024"),
        (era5_regridded - ghforecast144, "October 7, 2024"),
        (era5_regridded - ghforecast240, "October 10, 2024")
    ]
    
    custom_cmap = ListedColormap(['white', 'peachpuff', 'sandybrown', 'peru', 'sienna'])
    lon = ds2.longitude.values
    lat = ds2.latitude.values
    dataproj = ccrs.PlateCarree()
    
    for diff, date in differences:
        fig = plt.figure(figsize=(12,9))
        ax = fig.add_subplot(111, projection=ccrs.LambertConformal())
        ax.set_extent([-117,-75,20,55], ccrs.PlateCarree())
        ax.add_feature(cfeature.COASTLINE.with_scale('50m'))
        ax.add_feature(cfeature.STATES.with_scale('50m'), linestyle=':')
        ax.add_feature(cfeature.BORDERS.with_scale('50m'))
        
        cs = ax.contour(lon, lat, diff, colors='k', transform=dataproj)
        plt.clabel(cs)
        color_map = ax.contourf(lon, lat, diff, cmap=custom_cmap, transform=dataproj)
        ax.set_title(f'Geopotential Height Difference vs ERA5 (m), {date}')
        plt.colorbar(color_map, ax=ax)
        plt.show()

In [ ]:
run_date = "2024-10-02"
forecast_hours = [0, 72, 144, 240]
verify_with_era5(run_date, forecast_hours, era5_data_dir)